In [1]:

import pdb
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from os import getcwd


In [2]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
nltk.download('twitter_samples')

[nltk_data] Error loading twitter_samples: <urlopen error [Errno
[nltk_data]     11001] getaddrinfo failed>


False

In [4]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [5]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [6]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [7]:
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [8]:

import re
import string
from nltk.stem import PorterStemmer

def process_tweet(tweet):
    tweet = tweet.lower()
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  
            word not in string.punctuation):
            stem_word = stemmer.stem(word)  
            tweets_clean.append(stem_word)

    return tweets_clean


In [9]:
def count_tweets(result, tweets, ys):
 

    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):

            pair = (word, y)

            if pair in result:
                result[pair] += 1
            else:
                result[pair] = 1

    return result

In [10]:
freqs = count_tweets({}, train_x, train_y)

In [11]:
def lookup(freqs, word, label):

    n = 0 

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

In [12]:
def train_naive_bayes(freqs, train_x, train_y):

    loglikelihood = {}
    logprior = 0

 
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)


    N_pos = N_neg = 0
    for pair in freqs.keys():

        if pair[1] > 0:
            N_pos += freqs[pair]

        else:
            N_neg += freqs[pair]

    D = len(train_x)
    D_pos = sum(train_y)
    D_neg = D - D_pos

    logprior = np.log(D_pos)-np.log(D_neg)

    for word in vocab:
        freq_pos = lookup(freqs,word,1)
        freq_neg = lookup(freqs,word,0)

        p_w_pos = (freq_pos+1)/(N_pos + V)
        p_w_neg = (freq_neg +1)/(N_neg + V)

        loglikelihood[word] = np.log( p_w_pos/p_w_neg)

    return logprior, loglikelihood


In [13]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)

In [14]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    word_l = process_tweet(tweet)
    p = 0
    p += logprior

    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]
    return p


In [15]:
my_tweet = "she is a good dancer.I love her moves"
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The output is', p)
if(p>=0):
    print("positive sentiment")
else:
    print("negative sentiment")

The output is 0.952462057178133
positive sentiment


In [16]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    accuracy = 0  
    y_hats = []
    
    for tweet in test_x:     
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        y_hats.append(y_hat_i)

    error = np.mean(abs(y_hats- test_y))
    accuracy = 1-error

    return accuracy


In [17]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.9940
